In [48]:
import numpy as np
import dadi, random, pickle
import os, sys
sys.path.append(os.path.join(os.getcwd(), '..')) # this is the ml_dadi dir
import data_manip, ml_models
from data_manip import generating_data
from ml_models import rfr_train, mlpr_train

In [49]:
# generate a list of theta values to run scaling and add variance
theta_list_test = [1,10000,1000,100] # order of increase variance
theta_list_train = [1] 

In [50]:
# designate demographic model, sample size, and extrapolation grid 
func = dadi.Demographics1D.growth
ns = [160] #############################sample size (values on axis), do what we did to theta list, try different values 10, 20, 40, 80, 160; no rfr
pts_l = [40, 50, 60]

In [51]:
# designate which param to be in log scale
logs = [True, False] #in this case, nu is on log scale, T is not

In [52]:
 # Generate parameter list for training: exclude params where T/nu > 5 version
 # using log scale for nu
 # nu, T is the key to the data dictionary
train_params = [(nu,T) for nu in np.linspace(-2, 2, 25) # (lower, upper, number of values); linspace equally spaces values
                       for T in np.linspace(0.1, 2, 24)] #if T/10**nu <= 5] 

 # print training set info 
 print('n_samples training: ', len(train_params))
 print('Range of training params:', min(train_params), 'to', max(train_params))
 print('Theta list:', theta_list_train)
 # Make a list of training data dictionaries, one dictionary for each theta case
 list_train_dict=generating_data(train_params, theta_list_train, func, ns, pts_l, logs) #generating_data is a function that uses the given info to generate an fs; the key to this fs is nu and T
 pickle.dump(list_train_dict, open('data/train_data_ns_160', 'wb'), 2) ################################

n_samples training:  600
Range of training params: (-2.0, 0.1) to (2.0, 2.0)
Theta list: [1]


In [53]:
# Load new datasets for training
list_train_dict = pickle.load(open('data/train_data_ns_160','rb')) #########################only needed if we havent ran the previous block of code, i.e. if we haven't used list_train_dict before

In [54]:
# Train MLPR with adam solver (default)
# Also test this one
list_mlpr_adam = [mlpr_train(train_dict, max_iter=5000) # too high maxiter= runs too long, too low= non convergence
                        for train_dict in list_train_dict]
pickle.dump(list_mlpr_adam, open('data/list_mlpr_adam_ns_160', 'wb'), 2) #########################

In [20]:
# Train MLPR with lbfgs solver
#This will probably work the best
# Need large max_iter and take longer to run but perform the best for two_epoch
list_mlpr_lbfgs = [mlpr_train(train_dict, solver='lbfgs', max_iter=10000)
                        for train_dict in list_train_dict]
#mlpr_1 = mlpr_train(list_train_dict[2], solver='lbfgs', max_iter=10000)  # to test an individual theta value, uncomment this, comment out above
pickle.dump(list_mlpr_lbfgs, open('data/list_mlpr_lbfgs_ns_10', 'wb'), 2)

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.htmllbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html

In [55]:
# Generate Test Datasets
test_params = []
while len(test_params) < 100: 
# generate random nu and T within the same range as training data range
    nu = random.random() * 4 - 2 # nu in log scale
    T = random.random() * 1.9 + 0.1
    # exclude T/nu > 5
    if T/10**nu <= 5: # only appends to list if this condition is satisfied; different from below
        params = (nu, T)
        test_params.append(params)
# print testing set info 
print('n_samples testing: ', len(test_params))
print('Range of testing params:', min(test_params), 'to', max(test_params))
print('Theta list:', theta_list_test)
# Make a list of test data dictionaries, one dictionary for each theta case
list_test_dict = generating_data(test_params, theta_list_test, func, ns, pts_l, logs)
# Save testing set as a pickle file
pickle.dump(list_test_dict, open('data/test_data_ns_160', 'wb'), 2) ################################

n_samples testing:  100
Range of testing params: (-1.1183195768851477, 0.2444060527496501) to (1.9467636371024035, 1.4501354314447392)
Theta list: [1, 10000, 1000, 100]
